In [ ]:
from VR_Trajectory_analysis import *
RUN_HIDDEN_CELLS = False  # Change to True when you need to run hidden cells

In [ ]:
if RUN_HIDDEN_CELLS:
    directory1 = '/Users/apaula/ownCloud/MatrexVR1/20250715_GrayPreference/20250716_GrayPreference_V1_Data/RunData'
    directory2 = '/Users/apaula/ownCloud/MatrexVR3/202507115_GrayPreference/20250716_GrayPreference_V1_Data/RunData'

In [ ]:
if RUN_HIDDEN_CELLS:
    df1 = get_combined_df(directory1, trim_seconds=1.0)
    df2 = get_combined_df(directory2, trim_seconds=1.0)

    concat_df = pd.concat([df1, df2], ignore_index=True)

In [ ]:
if RUN_HIDDEN_CELLS:
    concat_df = add_trial_id_and_displacement(concat_df)
    concat_df = add_trial_time(concat_df)

In [ ]:
if RUN_HIDDEN_CELLS:
    concat_df_path_pickle = '/Users/apaula/src/VRDataAnalysis/Ants/preprocessed_data/staticDecisions/GreyPreferences_concat_df.pkl'
    concat_df.to_pickle(concat_df_path_pickle)

In [ ]:
concat_df_path_pickle = '/Users/apaula/src/VRDataAnalysis/Ants/preprocessed_data/staticDecisions/GreyPreferences_concat_df.pkl'
concat_df = pd.read_pickle(concat_df_path_pickle)

In [ ]:
df_stationary, df_normal, df_excessive, stationary_ids, normal_ids, excessive_ids = classify_trials_by_path_length(concat_df[(concat_df['stepName'] != 'skybox') & (concat_df['TotalDisplacement'].gt(12))], min_length=0, max_length=200)

In [ ]:
total_unique_trials = concat_df[(concat_df['stepName'] != 'skybox')]['UniqueTrialID'].nunique()

percent_stationary = df_stationary['UniqueTrialID'].nunique() / total_unique_trials * 100
percent_normal = df_normal['UniqueTrialID'].nunique() / total_unique_trials * 100
percent_excessive = df_excessive['UniqueTrialID'].nunique() / total_unique_trials * 100

percent_stationary, percent_normal, percent_excessive


In [ ]:
df_normal.columns

In [ ]:
df_normal['stepName'].unique()

In [ ]:
concat_df[concat_df['TotalDisplacement'].between(0.1, 5)]['TotalDisplacement'].hist(bins=30, color='gray', alpha=0.7)

In [ ]:
df_normal['UniqueTrialID'].nunique()*10/60/60/8

In [ ]:

#add angle column to df_joined
df_normal['Angle'] = df_normal['stepName'].apply(lambda x: parse_angle_from_config(x, default_angle=999.0))
# -----------------------------------------------------------------
#  Trajectories for all configs
#  shown in a grid, one panel per Angle
# -----------------------------------------------------------------
import math, re
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import gridspec

# 1.  The joined dataframe is assumed to exist:  df_normal



df_filt = df_normal

# make sure Angle is numeric so the sort works
df_filt['Angle'] = pd.to_numeric(df_filt['Angle'], errors='coerce')

angles = sorted(df_filt['Angle'].dropna().unique())
if not angles:
    raise ValueError("No matching configs with an Angle column found.")

# 3.  Build grid layout -----------------------------------------------------
n_panels = len(angles)
n_cols   = 3                               # change to taste
n_rows   = math.ceil(n_panels / n_cols)

fig = plt.figure(figsize=(n_cols*8, n_rows*7.5))
gs  = gridspec.GridSpec(n_rows, n_cols, wspace=.25, hspace=.35)

# 4.  One sub-plot per Angle -------------------------------------------------
for idx, angle in enumerate(angles):
    ax = fig.add_subplot(gs[idx])
    g  = df_filt[df_filt['Angle'] == angle]

    # trajectories
    for trial_id, td in g.groupby('UniqueTrialID'):
        ax.plot(td['GameObjectPosX'], td['GameObjectPosZ'], alpha=.1)

    # basic aesthetics
    #ax.set_xlim(-40, 40)
    #ax.set_ylim(-10, 70)
    ax.set_aspect('equal', adjustable='box')

    #ax.set_xticks([-60, -40, -20, 0, 20, 40, 60])
    #ax.set_yticks([0, 20, 40, 60])
    ax.tick_params(labelsize=8)

    ax.set_title(f"{int(angle)}°", fontsize=10)


# hide any leftover empty cells
for j in range(idx+1, n_rows*n_cols):
    fig.add_subplot(gs[j]).axis('off')

fig.suptitle("Trajectories – Grey Preferences", y=.92, fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
df_normal.columns

In [ ]:
df_normal['stepName'].unique()

In [ ]:
import re

def parse_step_name(name):
    match = re.search(r'_(\d{2})_(\d{2})_(\d{2,3})deg$', name)
    if match:
        shade1, shade2, angle = match.groups()
        return int(shade1), int(shade2), int(angle)
    else:
        return None, None, None

df_normal[['shade1', 'shade2', 'conflict_angle']] = df_normal['stepName'].apply(
    lambda x: pd.Series(parse_step_name(x))
)
trial_summary = (
    df_normal
      .sort_values(['UniqueTrialID', 'elapsed_time'])
      .groupby('UniqueTrialID')
      .agg(
          FlyID=('FlyID', 'first'),
          stepName=('stepName', 'first'),
          conflict_angle=('conflict_angle', 'first'),
          shade1=('shade1', 'first'),
          shade2=('shade2', 'first'),
          travel_dir=('TravelDirectionDeg', 'first')
      )
      .dropna(subset=['travel_dir'])
      .reset_index()
)

# Convert to signed
trial_summary['travel_dir_signed'] = (
    (trial_summary['travel_dir'] + 180) % 360 - 180
)
def extract_non_40_shade(row):
    if row['shade1'] == 40 and row['shade2'] == 40:
        return [40]  # will duplicate later
    elif row['shade1'] == 40:
        return [row['shade2']]
    elif row['shade2'] == 40:
        return [row['shade1']]
    else:
        return []  # exclude anything else?

# Expand rows with multiple non-40 values
trial_summary['non_40_shade'] = trial_summary.apply(extract_non_40_shade, axis=1)
trial_summary = trial_summary.explode('non_40_shade').dropna(subset=['non_40_shade'])
def get_40_side(row):
    if row['shade1'] == 40 and row['shade2'] == 40:
        return 'both'
    elif row['shade1'] == 40:
        return 'shade1'
    elif row['shade2'] == 40:
        return 'shade2'
    else:
        return 'none'

trial_summary['forty_side'] = trial_summary.apply(get_40_side, axis=1)


In [ ]:
trial_summary['stepName'].value_counts()


In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 2, figsize=(12, 10), sharey=True)
angle_order = [90, 180]
side_order = ['shade1', 'shade2']

for i, angle in enumerate(angle_order):
    for j, side in enumerate(side_order):
        ax = axes[i, j]
        subset = trial_summary[
            (trial_summary['conflict_angle'] == angle) &
            ((trial_summary['forty_side'] == side) | (trial_summary['forty_side'] == 'both'))
        ]
        ax.scatter(subset['non_40_shade'], subset['travel_dir_signed'], 
                   color='black', alpha=0.1, s=30)
        ax.axhline(0, color='grey', ls='--', lw=0.5)
        ax.set_title(f'Angle: {angle}°, 40° in {side}')
        ax.set_xlabel('Other shade of grey')
        ax.set_ylim(-180, 180)
        ax.set_yticks(range(-180, 181, 60))

axes[0,0].set_ylabel('Travel direction (signed deg)')
axes[1,0].set_ylabel('Travel direction (signed deg)')
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define bins
x_bins = sorted(trial_summary['non_40_shade'].dropna().unique())
y_bins = np.linspace(-180, 180, 73)  # 10° bins (adjustable)

fig, axes = plt.subplots(2, 2, figsize=(12, 10), sharex=True, sharey=True)
angle_order = [90, 180]
side_order = ['shade1', 'shade2']

for i, angle in enumerate(angle_order):
    for j, side in enumerate(side_order):
        ax = axes[i, j]

        # Filter data for this subplot
        subset = trial_summary[
            (trial_summary['conflict_angle'] == angle) &
            ((trial_summary['forty_side'] == side) | (trial_summary['forty_side'] == 'both'))
        ]

        # Prepare 2D histogram matrix
        hist = np.zeros((len(y_bins) - 1, len(x_bins)))

        for xi, shade in enumerate(x_bins):
            shade_data = subset[subset['non_40_shade'] == shade]['travel_dir_signed']
            hist[:, xi], _ = np.histogram(shade_data, bins=y_bins)

        # Show as image
        im = ax.imshow(
            hist, aspect='auto', origin='lower',
            extent=[min(x_bins) - 5, max(x_bins) + 5, -180, 180],
            cmap='viridis'
        )

        ax.set_title(f'Angle: {angle}°, 40° in {side}')
        ax.set_xlabel('Other shade of grey')
        ax.set_yticks(range(-180, 181, 60))

axes[0,0].set_ylabel('Travel direction (signed deg)')
axes[1,0].set_ylabel('Travel direction (signed deg)')
plt.tight_layout()
plt.colorbar(im, ax=axes, orientation='vertical', label='Count')
plt.show()


In [ ]:
# Copy to avoid mutating original
df_mirror = trial_summary.copy()

# Mirror the travel direction for shade2 trials
df_mirror['travel_dir_mirrored'] = df_mirror.apply(
    lambda row: -row['travel_dir_signed'] if row['forty_side'] == 'shade2' else row['travel_dir_signed'],
    axis=1
)

# Only keep trials where 40 is in shade1, shade2 (mirrored), or both
df_mirror = df_mirror[df_mirror['forty_side'].isin(['shade1', 'shade2', 'both'])]

# Define bins
x_bins = sorted(df_mirror['non_40_shade'].dropna().unique())
y_bins = np.linspace(-180, 180, 37)  # 10° bins

# One plot per angle now
fig, axes = plt.subplots(1, 2, figsize=(12, 5), sharey=True)
angle_order = [90, 180]

for i, angle in enumerate(angle_order):
    ax = axes[i]

    subset = df_mirror[df_mirror['conflict_angle'] == angle]

    # Initialize histogram matrix
    hist = np.zeros((len(y_bins) - 1, len(x_bins)))

    for xi, shade in enumerate(x_bins):
        shade_data = subset[subset['non_40_shade'] == shade]['travel_dir_mirrored']
        hist[:, xi], _ = np.histogram(shade_data, bins=y_bins)

        # Normalize each column (shade) by its own max
    #hist_max = hist.max(axis=0, keepdims=True)
    #hist_max[hist_max == 0] = 1  # Avoid division by zero
    #hist /= hist_max
    
    im = ax.imshow(
        hist, aspect='auto', origin='lower',
        extent=[min(x_bins) - 5, max(x_bins) + 5, -180, 180],
        cmap='viridis'
    )

    ax.set_title(f'Conflict angle: {angle}°')
    ax.set_xlabel('2nd shade of grey')
    ax.set_yticks(range(-180, 181, 60))
    ax.grid(False)

axes[0].set_ylabel('Mirrored travel direction (signed deg)')
plt.tight_layout()
plt.colorbar(im, ax=axes, orientation='vertical', label='Count')
plt.show()


In [ ]:
# Assign preference (1 = shade 2, 0 = shade 1)
df_mirror['shade2_choice'] = df_mirror['travel_dir_mirrored'] > 0

from statsmodels.stats.proportion import proportion_confint

# Step 1: Group and calculate counts and proportions
pref_summary = (
    df_mirror
    .groupby(['conflict_angle', 'non_40_shade'])
    .agg(
        n=('shade2_choice', 'count'),
        proportion=('shade2_choice', 'mean')
    )
    .reset_index()
)

# Step 2: Calculate Wilson confidence intervals for binomial proportion
# Note: need number of "successes" = proportion * n
pref_summary['ci_low'], pref_summary['ci_high'] = zip(*pref_summary.apply(
    lambda row: proportion_confint(count=int(row['proportion'] * row['n']),
                                   nobs=int(row['n']),
                                   method='wilson'),
    axis=1
))


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5), sharey=True)
angle_order = [90, 180]

for i, angle in enumerate(angle_order):
    ax = axes[i]

    subset = pref_summary[pref_summary['conflict_angle'] == angle]

    # Plot bar centers
    ax.bar(subset['non_40_shade'], subset['proportion'], width=5, color='black', alpha=0.7)

    # Plot error bars
    ax.errorbar(
        subset['non_40_shade'],
        subset['proportion'],
        yerr=[subset['proportion'] - subset['ci_low'], subset['ci_high'] - subset['proportion']],
        fmt='none',
        ecolor='red',
        capsize=3,
        elinewidth=1,
    )

    ax.axhline(0.5, color='gray', ls='--')  # neutral line
    ax.set_title(f'Preference Index — Conflict angle {angle}°')
    ax.set_xlabel('Other shade of grey')
    ax.set_ylim(0, 1)
    ax.set_xticks(sorted(df_mirror['non_40_shade'].unique()))
    ax.set_yticks([0, 0.25, 0.5, 0.75, 1])
    if i == 0:
        ax.set_ylabel('Proportion choosing shade 2')

plt.tight_layout()
plt.show()


In [ ]:
# Step 2: Group by FlyID, conflict angle, and shade
fly_pref = (
    df_mirror
    .groupby(['FlyID', 'conflict_angle', 'non_40_shade'])
    .agg(
        proportion=('shade2_choice', 'mean'),
        n=('shade2_choice', 'count')
    )
    .reset_index()
)
from statsmodels.stats.proportion import proportion_confint

# Calculate Wilson CIs for each fly's dot
fly_pref['ci_low'], fly_pref['ci_high'] = zip(*fly_pref.apply(
    lambda row: proportion_confint(count=int(row['proportion'] * row['n']),
                                   nobs=int(row['n']),
                                   method='wilson'),
    axis=1
))


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

sns.set(style='whitegrid')

# Get sorted numeric list of shades
x_shades = sorted(fly_pref['non_40_shade'].dropna().unique())

fig, axes = plt.subplots(1, 2, figsize=(12, 5), sharey=True)
angle_order = [90, 180]

for i, angle in enumerate(angle_order):
    ax = axes[i]
    subset = fly_pref[fly_pref['conflict_angle'] == angle]

    # Convert x values to numeric positions
    x_pos_map = {shade: xi for xi, shade in enumerate(x_shades)}
    subset['x_pos'] = subset['non_40_shade'].map(x_pos_map)

    # Plot individual dots with jitter around numeric positions
    jitter_strength = 0.15
    jittered_x = subset['x_pos'] + np.random.uniform(-jitter_strength, jitter_strength, size=len(subset))

    ax.scatter(
        jittered_x,
        subset['proportion'],
        alpha=0.5,
        s=30,
        color='black'
    )

    # Compute group-level stats
    group_stats = (
        subset.groupby('non_40_shade')
              .agg(mean=('proportion', 'mean'),
                   sem=('proportion', lambda x: x.std() / (len(x)**0.5)))
              .reset_index()
    )
    group_stats['x_pos'] = group_stats['non_40_shade'].map(x_pos_map)

    # Overlay error bars
    ax.errorbar(
        group_stats['x_pos'],
        group_stats['mean'],
        yerr=group_stats['sem'],
        fmt='o',
        color='red',
        capsize=4,
        lw=1
    )

    # Set axis labels and ticks
    ax.set_xticks(range(len(x_shades)))
    ax.set_xticklabels(x_shades)
    ax.axhline(0.5, color='gray', ls='--')
    ax.set_title(f'Conflict angle: {angle}°')
    ax.set_xlabel('Other shade of grey')
    if i == 0:
        ax.set_ylabel('Ant-level preference for shade 2')
    else:
        ax.set_ylabel('')
    ax.set_ylim(0, 1)

plt.tight_layout()
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

sns.set(style='whitegrid')

x_shades = sorted(fly_pref['non_40_shade'].dropna().unique())
x_pos_map = {shade: xi for xi, shade in enumerate(x_shades)}

fig, axes = plt.subplots(1, 2, figsize=(12, 5), sharey=True)
angle_order = [90, 180]

for i, angle in enumerate(angle_order):
    ax = axes[i]
    subset = fly_pref[fly_pref['conflict_angle'] == angle].copy()
    subset['x_pos'] = subset['non_40_shade'].map(x_pos_map)

    # Jitter
    jitter_strength = 0.15
    subset['jittered_x'] = subset['x_pos'] + np.random.uniform(-jitter_strength, jitter_strength, size=len(subset))

    # Plot individual dots, colored by number of trials
    sc = ax.scatter(
        subset['jittered_x'],
        subset['proportion'],
        c=subset['n'],
        s=subset['n'],             # size = number of trials
        cmap='viridis',            # or 'plasma', 'magma', etc.
        alpha=0.7,
        edgecolor='k',
        linewidth=0.3
    )

    # Group-level stats (optional overlay)
    group_stats = (
        subset.groupby('non_40_shade')
              .agg(mean=('proportion', 'mean'),
                   sem=('proportion', lambda x: x.std() / (len(x)**0.5)))
              .reset_index()
    )
    group_stats['x_pos'] = group_stats['non_40_shade'].map(x_pos_map)

    ax.errorbar(
        group_stats['x_pos'],
        group_stats['mean'],
        yerr=group_stats['sem'],
        fmt='o',
        color='black',
        capsize=4,
        lw=1,
        alpha=0.4
    )

    ax.set_xticks(range(len(x_shades)))
    ax.set_xticklabels(x_shades)
    ax.axhline(0.5, color='gray', ls='--')
    ax.set_title(f'Conflict angle: {angle}°')
    ax.set_xlabel('2nd shade of grey')
    if i == 0:
        ax.set_ylabel('Ant-level preference for 2nd shade')
    ax.set_ylim(0, 1)

# Create colorbar axis to the right of all subplots
from mpl_toolkits.axes_grid1 import make_axes_locatable

divider = make_axes_locatable(axes[-1])
cax = divider.append_axes("right", size="5%", pad=0.05)

cbar = plt.colorbar(sc, cax=cax)
cbar.set_label('Number of trials (per Ant)')

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Grey levels
greys = [0.0, 0.2, 0.4, 0.6, 0.8]
n = len(greys)

# Create the figure and axis
fig, ax = plt.subplots(figsize=(8, 2.8))

# Vertical positions
top_y = 1.1  # Top row y-position
bottom_y = 0  # Bottom rectangle y-position
rect_height = 1  # Height of both rows

# Top row: individual grey rectangles
for i, g in enumerate(greys):
    ax.add_patch(plt.Rectangle((i, top_y), 1, rect_height, color=str(g)))
    ax.text(i + 0.5, top_y + rect_height / 2, str(g),
            va='center', ha='center',
            fontsize=12, color='white' if g < 0.5 else 'black')

# Bottom rectangle: same total width, centered label
bottom_shade = 0.4
ax.add_patch(plt.Rectangle((0, bottom_y), n, rect_height, color=str(bottom_shade)))
ax.text(n / 2, bottom_y + rect_height / 2, str(bottom_shade),
        va='center', ha='center',
        fontsize=12, color='white' if bottom_shade < 0.5 else 'black')

# Set limits and remove axes
ax.set_xlim(0, n)
ax.set_ylim(0, top_y + rect_height + 0.1)
ax.axis('off')

plt.show()
